# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "https://raw.githubusercontent.com/kshedden/statswpy/master/NHANES/merged/nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU", 'BPXDI1']
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1: 

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [2]:
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()

x = pd.DataFrame({"GEE_params": result.params, "GEE_SE": result.bse})
x = x[["GEE_params", "GEE_SE"]]
print(x)

           GEE_params    GEE_SE
Intercept   70.008821  0.418665


In [3]:
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.031


__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


The ICC for diastolic blood pressure is 0.031. The systolic blood pressure ICC is 0.030. These numbers are small but not negligible, and the consistency suggests a cluster effect. This will make standard errors of an OLS or GLM model inaccurate.

## Question 2: 

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [5]:
model = sm.GEE.from_formula("BPXDI1 ~ RIAGENDR + RIDAGEYR + BMXBMI", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()

x = pd.DataFrame({"GEE_params": result.params, "GEE_SE": result.bse})
x = x[["GEE_params", "GEE_SE"]]
print(x)

           GEE_params    GEE_SE
Intercept   70.785818  1.097640
RIAGENDR    -2.742236  0.401613
RIDAGEYR    -0.040927  0.014387
BMXBMI       0.183877  0.029573


In [6]:
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

The ICC for this model is 0.030. The cluster effect is small and robust, with precise consistency across variables. Because the cluster effect did not make a notable change, we can conclude the covariates either are evenly distributed across clusters or their correlation to diastolic blood pressure is quite low.

## Question 3: 

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [7]:
female = da[da.RIAGENDR == 2]
male = da[da.RIAGENDR == 1]

In [8]:
#GEE Female
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=female)
result = model.fit()

x = pd.DataFrame({"GEE_params": result.params, "GEE_SE": result.bse})
x = x[["GEE_params", "GEE_SE"]]
print(x)
print(result.cov_struct.summary())

           GEE_params    GEE_SE
Intercept   68.772111  0.401327
The correlation between two observations in the same cluster is 0.029


In [9]:
#GEE Male
model = sm.GEE.from_formula("BPXDI1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=male)
result = model.fit()

x = pd.DataFrame({"GEE_params": result.params, "GEE_SE": result.bse})
x = x[["GEE_params", "GEE_SE"]]
print(x)
print(result.cov_struct.summary())

           GEE_params   GEE_SE
Intercept   71.253287  0.52976
The correlation between two observations in the same cluster is 0.035


__Q3a.__ What do you learn by comparing these two fitted models?

Gender is not evenly distributed across clusters and is correlated to diastolic blood pressure. Men have a higher diastolic blood pressure, on average.

## Question 4: 

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [10]:
#The statsmodel website informs that
#statsmodels.regression.mixed_linear_model.MixedLM.from_formula
#defaults with a random intercept for groups.

model = sm.MixedLM.from_formula("BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI + C(RIDRETH1)",
           groups="group", data=da)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:              MixedLM Dependent Variable: BPXDI1     
No. Observations:   5102    Method:             REML       
No. Groups:         30      Scale:              153.0915   
Min. group size:    106     Log-Likelihood:     -20100.7500
Max. group size:    226     Converged:          Yes        
Mean group size:    170.1                                  
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        68.717    1.202 57.170 0.000 66.361 71.073
C(RIDRETH1)[T.2]  1.505    0.697  2.160 0.031  0.139  2.871
C(RIDRETH1)[T.3]  0.512    0.597  0.858 0.391 -0.658  1.682
C(RIDRETH1)[T.4]  2.537    0.659  3.852 0.000  1.246  3.828
C(RIDRETH1)[T.5]  3.491    0.690  5.061 0.000  2.139  4.843
RIDAGEYR         -0.036    0.010 -3.590 0.000 -0.056 -0.016
RIAGENDR         -2.750    0.349 -7.883 0.000 -3.433 -2.066
BMXBMI            0.197    0.026  7.642 0.000  0.147  0.248
group Var         4.196    0.110                           
===========================================================

"""

__Q4a.__ How would you describe the strength of the clustering in this analysis?

The variance for groups is estimated at 4.196. If we were to choose two groups at random, their random effects would differ on average by around 2.90 (calculated as sqrt(4.196 * 2). This is a sizeable shift, comparable to near 15 BMI towards diastolic blood pressure modeling.

__Q4b:__ Include a random intercept for age, and describe your findings.

In [11]:
da["age_cen"] = da.groupby("group").RIDAGEYR.transform(lambda x: x - x.mean())

In [12]:
model = sm.MixedLM.from_formula("BPXDI1 ~ age_cen + RIAGENDR + BMXBMI + C(RIDRETH1)",
           groups="group", vc_formula={"age_cen": "0+age_cen"}, data=da)
result = model.fit()
result.summary()

C:\Users\tiesa\anaconda3\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
===========================================================
Model:              MixedLM Dependent Variable: BPXDI1     
No. Observations:   5102    Method:             REML       
No. Groups:         30      Scale:              156.3896   
Min. group size:    106     Log-Likelihood:     -20141.0497
Max. group size:    226     Converged:          Yes        
Mean group size:    170.1                                  
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        67.181    1.025 65.565 0.000 65.173 69.189
C(RIDRETH1)[T.2] -0.008    0.637 -0.013 0.990 -1.256  1.240
C(RIDRETH1)[T.3]  0.401    0.524  0.765 0.444 -0.626  1.427
C(RIDRETH1)[T.4]  2.027    0.570  3.558 0.000  0.910  3.144
C(RIDRETH1)[T.5]  3.651    0.629  5.803 0.000  2.418  4.883
age_cen          -0.031    0.015 -2.102 0.036 -0.060 -0.002
RIAGENDR         -2.780    0.352 -7.890 0.000 -3.470 -2.089
BMXBMI            0.197    0.026  7.626 0.000  0.146  0.248
age_cen Var       0.003    0.000                           
===========================================================

"""

The estimated variance for random age slopes is 0.003, which translates to a standard deviation of 0.055. With a centered age coefficient at -0.031, we see that in some clusters diastolic blood pressure increases by 0.024 units per year while in others, it decreases by 0.086 units per year.